In [ ]:
from RAGLibrary import myWidgets, myRAG, checkConstruct, createSchema, faissConvert, embedding
import os
import json
import faiss
from typing import Any, Dict

In [ ]:
widgets_list = myWidgets.create_name_form()

In [ ]:
""" DEFINE """

data   = widgets_list[0] #HBox 1
keys   = widgets_list[1] #HBox 2
choose = widgets_list[2] #HBox 3

embedd_model = widgets_list[3]
search_egine = widgets_list[4]
rerank_model = widgets_list[5]
respon_model = widgets_list[6]
API_drop     = widgets_list[7]
button_box   = widgets_list[8]

# HBox 1
file_name = data.children[0]
file_type = data.children[1]

# HBox 2
data_key = keys.children[0]
embe_key = keys.children[1]

# HBox 3
switch_model = choose.children[0]
merge_otp    = choose.children[1]
path_end_val = choose.children[1]

# Get value
data_folder   = file_name.value
file_type_val = file_type.value

data_key_val  = data_key.value
embe_key_val  = embe_key.value

API_key_val = API_drop.value
switch      = switch_model.value
merge       = merge_otp.value
path_end    = path_end_val.value

embedding_model = embedd_model.value
searching_egine = search_egine.value
reranking_model = rerank_model.value
responing_model = respon_model.value


# Define
base_path = f"../Data/{data_folder}/{file_type_val}_{data_folder}"

json_file_path = f"{base_path}_Database.json"
schema_ex_path = f"{base_path}_Schema.json"
embedding_path = f"{base_path}_Embeds_{merge}"

torch_path  = f"{embedding_path}.pt"
faiss_path  = f"{embedding_path}.faiss"
mapping_path = f"{embedding_path}_mapping.json"
mapping_data = f"{embedding_path}_map_data.json"

FILE_TYPE    = file_type_val
DATA_KEY     = data_key_val
EMBE_KEY     = embe_key_val
SWITCH       = switch
EMBEDD_MODEL = embedding_model
SEARCH_EGINE = searching_egine
RERANK_MODEL = reranking_model
RESPON_MODEL = responing_model

if FILE_TYPE == "Data":
    MERGE = merge
else: 
    MERGE = "no_Merge"

API_KEY = API_key_val

SEARCH_ENGINE = faiss.IndexFlatIP

print("\n")
print(f"Embedder: {EMBEDD_MODEL}")
print(f"Searcher: {SEARCH_EGINE}")
print(f"Reranker: {RERANK_MODEL}")
print(f"Responer: {RESPON_MODEL}")
print(f"Data Key: {DATA_KEY}")
print(f"Embe Key: {EMBE_KEY}")
print(f"Database: {json_file_path}")
print(f"Torch   : {torch_path}")
print(f"Faiss   : {faiss_path}")
print(f"Mapping : {mapping_path}")
print(f"Map Data: {mapping_data}")
print(f"Schema  : {schema_ex_path}")
print(f"Model   : {SWITCH}")
print(f"Merge   : {MERGE}")
print(f"API Key : {API_KEY}")

In [ ]:
""" DATA TYPE """

def get_standard_type(value: Any) -> str:
    if isinstance(value, int):
        return "number"
    elif isinstance(value, float):
        return "number"
    elif isinstance(value, str):
        return "string"
    elif isinstance(value, bool):
        return "boolean"
    elif isinstance(value, list):
        return "array"
    elif isinstance(value, dict):
        return "object"
    elif value is None:
        return "null"
    return "unknown"

In [ ]:
""" EXTRACT SCHEMA """

def extract_schema(data: Any, prefix: str = "", schema: Dict[str, str] = None) -> Dict[str, str]:
    """Đệ quy phân tích cấu trúc JSON để tạo schema."""
    if schema is None:
        schema = {}
    
    if isinstance(data, dict):
        for key, value in data.items():
            new_prefix = f"{prefix}{key}" if prefix else key
            schema[new_prefix] = get_standard_type(value)
            if isinstance(value, dict):
                extract_schema(value, f"{new_prefix}.", schema)
            elif isinstance(value, list) and value:
                # Chỉ phân tích sâu nếu phần tử đầu tiên là dict hoặc list
                if isinstance(value[0], (dict, list)):
                    extract_schema(value[0], f"{new_prefix}.", schema)
    return schema

In [ ]:
""" CREATE SCHEMA"""

def create_schema(json_file_path: str, schema_path: str) -> Dict[str, str]:
    """Tạo schema từ tất cả các bộ JSON, kiểm tra trường đã xét và kiểu dữ liệu trả về."""
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Nếu không phải list, chuyển thành list
        if not isinstance(data, list):
            data = [data]
        
        # Kiểm tra nếu JSON rỗng
        if not data:
            raise ValueError("JSON data is empty")

        # Kiểm tra từng phần tử trong list, đảm bảo chúng là dict
        for i, item in enumerate(data, 1):
            if not isinstance(item, dict):
                print(f"Item {i} is not a dict: {type(item)}")

        processed_fields = set()
        full_schema = {}
        
        print("\nChecking fields and their types:")

        # Duyệt qua từng phần tử trong dữ liệu
        for i, item in enumerate(data, 1):
            if not isinstance(item, dict):
                print(f"Skipping item {i}: Not a dict ({type(item)})")
                continue

            # Trích xuất schema từ phần tử hiện tại
            schema = extract_schema(item)

            # Duyệt qua các trường trong schema
            for key, value in schema.items():
                # Nếu trường chưa có trong tập hợp thì lưu vào processed_fields
                if key not in processed_fields:
                    print(f"New: {key:15} item {i}: Type = {value}")
                    processed_fields.add(key)
                    
                # Cập nhật schema đầy đủ
                if key not in full_schema:
                    full_schema[key] = value

                # Xử lý trường hợp mixed với null
                elif full_schema[key] != value:
                    if value == "null":
                        # Nếu giá trị hiện tại là null, giữ nguyên kiểu cũ
                        print(f"Field '{key}' has null value in item {i}, keeping type: {full_schema[key]}")
                    elif full_schema[key] == "null":
                        # Nếu kiểu cũ là null, lấy kiểu mới (không phải null)
                        print(f"Field '{key}' has non-null value in item {i}, updating type to: {value}")
                        full_schema[key] = value
                    elif full_schema[key] != "mixed":
                        # Nếu không phải null và có sự khác biệt, đánh dấu là mixed
                        print(f"Warning: Field '{key}' has multiple types: {full_schema[key]} and {value}")
                        full_schema[key] = "mixed"
                           
        with open(schema_path, 'w', encoding='utf-8') as f:
            json.dump(full_schema, f, ensure_ascii=False, indent=2)
        
        # In schema
        print("Generated schema:")
        print(json.dumps(full_schema, ensure_ascii=False, indent=2))
        return full_schema
    
    except Exception as e:
        print(f"Error creating schema: {e}")
        return {}

In [ ]:
""""" MAIN - CREATE SCHEMA """""

if os.path.exists(json_file_path):
    create_schema(json_file_path, schema_ex_path)
else:
    print(f"JSON path does not exist: {json_file_path}")